https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#sphx-glr-download-tutorial-10-key-features-005-visualization-py

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] ="expandable_segments:True" # seems to stop gpu mem from filling up despite clearing

In [3]:
import torch
import pandas as pd
from pathlib import Path
import optuna
from reprpo.hp.helpers import optuna_df

In [4]:
from reprpo.training import train
from reprpo.experiments import experiment_configs
from reprpo.hp.space import search_spaces

[2024-10-08 21:11:00,686] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


## Objective

In [5]:
SEED=42
key_metric = "acc_gain_vs_ref/oos"
torch.manual_seed(SEED)
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
# silence please
import os
from loguru import logger
logger.remove()
logger.remove()
logger.add(os.sys.stderr, level="WARNING")

os.environ["WANDB_MODE"] = "disabled"
os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["TQDM_DISABLE"] = "true"

In [7]:
f_db = f"sqlite:///optuna.db"
f = f_db.replace('sqlite:///', './')
print(f)
Path(f).parent.mkdir(parents=True, exist_ok=True)
f_db

./optuna.db


'sqlite:///optuna.db'

In [8]:
# print(f'to visualise run in cli\ncd nbs\noptuna-dashboard {f_db}')

In [9]:
from reprpo.hp.target import override, default_tuner_kwargs, list2tuples, objective
import functools
from reprpo.experiments import experiment_configs
import copy
import wandb

import optuna.pruners
from optuna_integration.wandb import WeightsAndBiasesCallback

## Opt

Note on pruning. It's only really usefull with validation metrics and for long jobs over many epochs. I've got a small proxy job so there is no need.

In [10]:
# from reprpo.experiments import experiment_configs
from reprpo.hp.space import experiment_configs
experiment_configs.keys()

dict_keys(['hs-svd-mse', 'hs-hra-rank', 'hs-ortho-prefvec', 'ether-prefvec', 'dpo', 'projbp', 'projgrad3'])

In [11]:
import warnings
warnings.filterwarnings("ignore", category=optuna.exceptions.ExperimentalWarning) 

In [12]:
from optuna.study.study import storages, get_all_study_names
study_names = get_all_study_names(storage=f_db)

for study_name in study_names:
    print(study_name)
    study = optuna.load_study(study_name=study_name, storage=f_db)
    try:
        df_res = optuna_df(study, key_metric)
        display(df_res)
        print()
    except ValueError as e:
        print('-')

projgrad2


[W 2024-10-08 21:11:57,240] Study instance does not contain completed trials.


,importance,best
"projgrad2 N=✓0/315, best=nan",,



side-ether-prefvec


,importance,best
"side-ether-prefvec N=✓208/209, best=1.169",,
β,0.220,0.403787
use_angle_loss,0.171,True
use_dpo_loss,0.170,False
Htype,0.127,oft
lr,0.107,0.000615
weight_tokens,0.089,False
collect_input,0.028,False
use_orth_loss,0.024,False
reduction,0.020,25


...

,importance,best
"hs-hra-rank N=✓259/262, best=1.152",,
lr,0.899,0.000333
β,0.054,0.37992
r,0.042,38
apply_GS,0.005,True
α,0.000,0.280164



ether-prefvec


,importance,best
"ether-prefvec N=✓261/266, best=1.183",,
lr,0.844,0.000378
β,0.047,1.984854
reduction,0.031,1
nb,0.028,20
Htype,0.012,ether
use_proj_rel,0.010,True
use_dpo_loss,0.006,False
use_angle_loss,0.006,True
collect_hs,0.005,False



projgrad3


,importance,best
"projgrad3 N=✓140/140, best=1.279",,
lr,0.911,0.000232
β,0.046,0.842679
weight_dim,0.024,1
reverse_pref,0.006,True
neg_slope,0.005,0
scale_orth,0.005,False
mag_clip,0.003,float
mag_clip_value,NaN,0.230096


In [13]:
# unit test, moved to pytest
# for exp_name, (N, trial2args) in search_spaces.items():
#     study = optuna.create_study(direction="maximize")
#     cfg = copy.deepcopy(experiment_configs[exp_name][1])
#     print('exp_name', exp_name)
#     for _ in range(10):
#         trial = study.ask()
#         kwargs = trial2args(trial)
#         override(cfg, default_tuner_kwargs)
#         override(cfg, kwargs)
#         kwargs = list2tuples(kwargs)

#     # try one dev run
#     kwargs['dev'] = True
#     _objective = functools.partial(objective, key_metric=key_metric, starter_experiment_name=exp_name, trial2args=trial2args)

#     study.optimize(_objective, 
#                 n_trials=1, # do 20 at a time, round robin, untill done
#                 gc_after_trial=True, 
#                 # catch=(AssertionError, OSError, RuntimeError, KeyError, torch.OutOfMemoryError)
#     )

In [14]:
MAX_TRIALS= 250
import numpy as np
spaces = list(search_spaces.items())
while True:
    np.random.shuffle(spaces)
    for exp_name, (max_trials, trial2args) in spaces:
        try:
            study_name = f"{exp_name}"
            study = optuna.create_study(
                study_name=study_name,
                direction="maximize",
                load_if_exists=True,
                storage=f_db,
                sampler=optuna.samplers.TPESampler(seed=SEED),
                # pruner=optuna.pruners.NopPruner(),
            )

            n = 0
            try:
                df = study.trials_dataframe().sort_values('value', ascending=False)
                n = len(df)
            except Exception as e:
                print(e)
                pass
            if n>0:
                print(f"loaded {n} {study_name} trials")

                df_res = optuna_df(study, key_metric)
                print(df_res.to_markdown())

            
            if n < max_trials:
                _objective = functools.partial(objective, key_metric=key_metric, starter_experiment_name=exp_name, trial2args=trial2args)

                study.optimize(_objective, 
                            n_trials=20, # do 20 at a time, round robin, untill done
                            gc_after_trial=True, 
                            catch=(AssertionError, OSError, RuntimeError, KeyError, torch.OutOfMemoryError)
                )

            print('='*80)
        except KeyboardInterrupt:
            break
        except Exception as e:
            logger.exception(e)

[I 2024-10-08 21:12:26,362] Using an existing study with name 'projgrad3' instead of creating a new one.


loaded 140 projgrad3 trials
| projgrad3 N=✓140/140, best=1.279   |   importance | best                  |
|:-----------------------------------|-------------:|:----------------------|
| lr                                 |        0.886 | 0.0002320529194140666 |
| β                                  |        0.058 | 0.8426788311104311    |
| weight_dim                         |        0.038 | 1                     |
| scale_orth                         |        0.006 | False                 |
| mag_clip                           |        0.005 | float                 |
| neg_slope                          |        0.004 | 0                     |
| reverse_pref                       |        0.003 | True                  |
| mag_clip_value                     |      nan     | 0.2300957741124143    |


[I 2024-10-08 21:16:06,157] Trial 140 finished with value: 1.111753371868979 and parameters: {'lr': 0.00016063655684666597, 'β': 0.5100960908891031, 'reverse_pref': True, 'scale_orth': False, 'weight_dim': 0, 'neg_slope': 0, 'mag_clip': None}. Best is trial 119 with value: 1.2793834296724471.



| acc_inc/eval_ds [pp]                                    |   train |   test |    oos |   rnd |
|:--------------------------------------------------------|--------:|-------:|-------:|------:|
| ProjGrad lr=0.00016 neg_slope=0 scale_orth=False β=0.51 |   5.785 | -2.362 | 11.175 | 5.263 |


[I 2024-10-08 21:19:29,635] Trial 141 finished with value: 1.0385356454720618 and parameters: {'lr': 0.00019143905959604198, 'β': 0.8518384510960426, 'reverse_pref': True, 'scale_orth': False, 'weight_dim': 0, 'neg_slope': 0, 'mag_clip': 'float', 'mag_clip_value': 1059.0998275670322}. Best is trial 119 with value: 1.2793834296724471.


...


| acc_inc/eval_ds [pp]                                                 |   train |   test |    oos |     rnd |
|:---------------------------------------------------------------------|--------:|-------:|-------:|--------:|
| ProjGrad lr=0.00047 mag_clip=4.3 neg_slope=0 scale_orth=False β=0.91 |       0 | -4.724 | -0.771 | -12.281 |


[I 2024-10-08 21:29:33,178] Trial 144 finished with value: 1.0867052023121389 and parameters: {'lr': 0.00030898901095804786, 'β': 0.872733449727983, 'reverse_pref': True, 'scale_orth': False, 'weight_dim': 0, 'neg_slope': 0, 'mag_clip': 'float', 'mag_clip_value': 531.6958107355589}. Best is trial 119 with value: 1.2793834296724471.



| acc_inc/eval_ds [pp]                                                     |   train |   test |   oos |    rnd |
|:-------------------------------------------------------------------------|--------:|-------:|------:|-------:|
| ProjGrad lr=0.00031 mag_clip=5.3e+02 neg_slope=0 scale_orth=False β=0.87 |   4.132 |      0 | 8.671 | 35.088 |


[I 2024-10-08 21:32:53,264] Trial 145 finished with value: 0.9633911368015414 and parameters: {'lr': 0.00038435384531192336, 'β': 0.8163076145812128, 'reverse_pref': True, 'scale_orth': False, 'weight_dim': 0, 'neg_slope': 0, 'mag_clip': 'float', 'mag_clip_value': 51.97942363336419}. Best is trial 119 with value: 1.2793834296724471.



| acc_inc/eval_ds [pp]                                                |   train |   test |    oos |   rnd |
|:--------------------------------------------------------------------|--------:|-------:|-------:|------:|
| ProjGrad lr=0.00038 mag_clip=52 neg_slope=0 scale_orth=False β=0.82 |   2.479 | -2.362 | -3.661 | 5.263 |


## plot

In [ ]:
# You can use Matplotlib instead of Plotly for visualization by simply replacing `optuna.visualization` with
# `optuna.visualization.matplotlib` in the following examples.
from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_rank
from optuna.visualization.matplotlib import plot_slice
from optuna.visualization.matplotlib import plot_timeline

In [ ]:
search_spaces.keys()

In [ ]:
exp_name = 'projgrad'
trial2args = search_spaces[exp_name]

study_name = f"{exp_name}"
study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    load_if_exists=True,
    storage=f_db,
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.NopPruner(),
)
print('study.best_trial', study.best_trial)
df = study.trials_dataframe().query('state == "COMPLETE"').sort_values('value', ascending=False)
print(len(df))
plot_optimization_history(study)

In [13]:
# plot_timeline(study)

In [29]:
# plot_intermediate_values(study)

In [30]:
# plot_contour(study)


In [ ]:
plot_slice(study)


In [ ]:
plot_param_importances(study)

In [ ]:
plot_intermediate_values(study)

In [ ]:
plot_parallel_coordinate(study)

### Apendix 1: dataclass 2 optuna

In [20]:
# import inspect
# import typing
# from typing import Literal

# def optuna_suggest_from_dataclass(t):
#     n = t.__name__
#     print(f'## {n}')
#     sig = inspect.signature(t)
#     for name, param in sig.parameters.items():
#         if param.annotation== bool:
#             print(f'"{name}": trial.suggest_categorical("{name}", [True, False]),')
#         elif param.annotation==int:
#             print(f'"{name}": trial.suggest_int("{name}", 1, 10),')
#         elif param.annotation ==float:
#             print(f'"{name}": trial.suggest_float("{name}", 0.1, 10.0),')
#         elif param.annotation == str:
#             print(f'"{name}": trial.suggest_categorical("{name}", ["a", "b", "c"]),')
#         elif param.annotation == tuple:
#             print(f'"{name}": trial.suggest_categorical("{name}", [(1, 2), (3, 4), (5, 6)]),')
#         elif typing.get_origin(param.annotation) == Literal:
#             print(f'"{name}": trial.suggest_categorical("{name}", {param.annotation.__args__}),')
#         else:
#             print(f"!!Unknown type {param}")
#             # print(name, param.default, param.annotation)

# optuna_suggest_from_dataclass(ReprPOConfig)
# for t in Transforms:
#     print(f'## {t}')
#     optuna_suggest_from_dataclass(t.value)
# for l in Losses:
#     print(f'## {l}')
#     optuna_suggest_from_dataclass(l.value)


# optuna_suggest_from_dataclass(DPOProjGradConfig)